## UNIX Password System

In [1]:
import hashlib
import json
import os
import uuid

password_db = {}
password_file_name = 'password_file'

Utility functions
- get_sha2() : get sha2 hash of the 'data'
- update_pwd_file() : update the user and password to file on disk
- read_pwd_file() : read all the user and password information from file on disk

In [2]:
def get_password_hash(pwd, salt):
    hash_object = hashlib.sha256(salt.encode() + pwd.encode())
    hex_digest = hash_object.hexdigest()
    # print(hex_digest)
    return hex_digest


def update_pwd_file(pwd_data):
    with open(password_file_name, 'w') as outfile:
        json.dump(pwd_data, outfile)


def read_pwd_file():
    with open(password_file_name) as data_file:
        try:
            pwd_data = json.load(data_file)
        except Exception:
            pwd_data = {}
    return pwd_data


In [3]:
# Function to check if the user's entered password is correct for the user to sign in
def authenticate(user='', pwd='', direct_verify=False):
    if not direct_verify:
        user = input("Enter username: ")

        if user not in password_db:
            print("Error: User does not exist. Try signing up!")
            return False

        pwd = input("Enter password: ")

    pwd_hash = get_password_hash(pwd, password_db[user]['salt'])
    if password_db[user]['pwd_hash'] == pwd_hash:
        print("Password verified. Sign in successful!")
        return True
    else:
        print("Incorrect Password!")
        return False

In [4]:
# Function to Sign up : creates new user/password entry in the password file
def create_user_entry():
    user = input("Enter username: ")
    if user in password_db:
        print("Error: User Already exists")
        return False

    pwd = input("Enter password: ")

    salt = uuid.uuid4().hex

    pwd_hash = get_password_hash(pwd, salt)
    password_db[user] = {}
    password_db[user]['pwd_hash'] = pwd_hash
    password_db[user]['salt'] = salt
    update_pwd_file(password_db)
    print("Created New User Entry: {}".format(user))

In [5]:
# Change Password: Update the user's password (if the current password is correct)
def change_password():
    user = input("Enter username: ")

    if user not in password_db:
        print("Error: User does not exist!")
        return False

    old_password = input("Enter current password: ")
    new_password = input("Enter new password: ")

    if not authenticate(user, old_password, direct_verify=True):
        print("Error: Incorrect Current Password")
        return False

    salt = uuid.uuid4().hex
    pwd_hash = get_password_hash(new_password, salt)
    password_db[user]['pwd_hash'] = pwd_hash
    password_db[user]['salt'] = salt

    update_pwd_file(password_db)
    print("Password successfully changed for user: {}".format(user))

In [ ]:
def interactive_console():
    global password_db
    password_db = read_pwd_file()
    print("Welcome to ENPM693 Authentication System ")
    while True:
        print("***********************")
        print("1.Sign Up")
        print("2.Sign in")
        print("3.Change Password")
        option_number = input("Select Option Number:")
        if option_number not in ['1', '2', '3']:
            print("Incorrect option. Try again!\n")
            continue
        option_number = int(option_number)
        options = {1: create_user_entry,
                   2: authenticate,
                   3: change_password,
                   }
        options[option_number]()


if __name__ == '__main__':
    # code to handle password file not being present
    if not os.path.exists(password_file_name):
        with open(password_file_name, 'w') as data_file:
            data_file.write('{}')

    interactive_console()

Welcome to ENPM693 Authentication System 
***********************
1.Sign Up
2.Sign in
3.Change Password
Select Option Number:1
Enter username: user1
Enter password: pass1
Created New User Entry: user1
***********************
1.Sign Up
2.Sign in
3.Change Password
Select Option Number:2
Enter username: user1
Enter password: pass1
Password verified. Sign in successful!
***********************
1.Sign Up
2.Sign in
3.Change Password
Select Option Number:3
Enter username: user1
Enter current password: pass1
Enter new password: pass2
Password verified. Sign in successful!
Password successfully changed for user: user1
***********************
1.Sign Up
2.Sign in
3.Change Password
Select Option Number:2
Enter username: user1
Enter password: pass2
Password verified. Sign in successful!
***********************
1.Sign Up
2.Sign in
3.Change Password
